In [112]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

1. Use the beautifulsoup library to scrape the data (from menuism.com like the link above) on state names and corresponding number of store locations, for the following chains:
Starbucks

Dunkin’ Donuts

Peet’s Coffee & Tea

Tim Horton’s

Panera Bread

Caribou Coffee

Au Bon Pain

The Coffee Bean & Tea Leaf

McDonald’s

In [132]:
import re
rows = []
response = requests.get("https://web.archive.org/web/20220518102911/https://www.menuism.com/restaurant-locations/dunkin-donuts-181624")
soup = BeautifulSoup(response.content, "html.parser")
for i in soup.find_all('a', title=re.compile("Starbucks Coffee locations in .*, us")):
    x = i.text.split(" Starbucks Coffee locations ")
    x[1] = x[1].strip('(').strip(')')
    rows.append({
        "State": x[0],
        "Store": "Starbucks",
        "Store Locations": x[1],
    })  


In [133]:
rows = []
response = requests.get("https://web.archive.org/web/20220824121240/https:/www.menuism.com/restaurant-locations/starbucks-coffee-39564")
soup = BeautifulSoup(response.content, "html.parser")
for i in soup.find_all('a', title=re.compile("Dunkin' Donuts locations in .*, us")):
    x = i.text.split(" Dunkin' Donuts locations ")
    x[1] = x[1].strip('(').strip(')')
    rows.append({
        "State": x[0],
        "Store": "Dunkin' Donuts",
        "Store Locations": x[1],
    })
    
    

""


In [114]:
x = soup.find_all('a', title=re.compile("Starbucks Coffee locations in .*, us"))[0].text.split(" Starbucks Coffee locations ")
x[1] = x[1].strip('(').strip(')')
x

['Alaska', '24']

In [138]:
coffee = pd.read_csv("df_lab4_scraped.csv")
coffee = coffee[['State', 'Coffee Chain', 'Location Count']]
coffee

,State,Coffee Chain,Location Count
0,Alaska,Starbucks,24
1,Alabama,Starbucks,73
2,Arkansas,Starbucks,33
3,Arizona,Starbucks,279
4,California,Starbucks,2362
...,...,...,...
264,Vermont,McDonald's,30
265,Washington,McDonald's,326
266,Wisconsin,McDonald's,353
267,West Virginia,McDonald's,107


2. Write a function stateabb() that takes a state name (assume it’s spelled correctly) and converts it to its state abbreviation. This can be a very simple function.

In [116]:
def stateabb(state_name):
    states = {'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'}
    return states.get(state_name)

3. Parse, merge and tidy your data so that you have a row for each state and two columns: state abbrevation, location count.

In [117]:
coffee['State Abbreviation'] = coffee['State'].apply(stateabb)
coffee.head()

,State,Coffee Chain,Location Count,State Abbreviation
0,Alaska,Starbucks,24,AK
1,Alabama,Starbucks,73,AL
2,Arkansas,Starbucks,33,AR
3,Arizona,Starbucks,279,AZ
4,California,Starbucks,2362,CA


In [118]:
len(coffee)

269

4. Scrape the state names and populations from this wikipedia page. Convert the state names to abbreviations and merge these data with your coffee dataset.

In [119]:
population = pd.read_html("https://simple.wikipedia.org/wiki/List_of_U.S._states_by_population", attrs = {'class': 'wikitable sortable', "style": "width:100%; text-align:center;"})[0]
pop = population[['State','Census population, April 1, 2020 [1][2]']]
pop = pop.rename(columns = {'Census population, April 1, 2020 [1][2]': 'Population'})
pop.head()



,State,Population
0,California,39538223
1,Texas,29145505
2,Florida,21538187
3,New York,20201249
4,Pennsylvania,13002700


In [120]:
pop['State'] = pop['State'].apply(stateabb)
pop.head()

,State,Population
0,CA,39538223
1,TX,29145505
2,FL,21538187
3,NY,20201249
4,PA,13002700


In [137]:
combined = coffee.merge(pop, how='left', left_on='State Abbreviation', right_on='State')
combined = combined[['State_x', 'Coffee Chain', 'Location Count', 'State Abbreviation', 'Population']].rename(columns={'State_x': 'State'})
combined

,State,Coffee Chain,Location Count,State Abbreviation,Population
0,Alaska,Starbucks,24,AK,733391
1,Alabama,Starbucks,73,AL,5024279
2,Arkansas,Starbucks,33,AR,3011524
3,Arizona,Starbucks,279,AZ,7151502
4,California,Starbucks,2362,CA,39538223
...,...,...,...,...,...
327,Vermont,McDonald's,30,VT,643077
328,Washington,McDonald's,326,WA,7705281
329,Wisconsin,McDonald's,353,WI,5893718
330,West Virginia,McDonald's,107,WV,1793716


In [136]:
len(combined)

332

5. Find the revenue, stock price, or your financial metric of choice for each of the companies listed above (if you can find a website to scrape these from that’s great!…but it’s okay if you manually enter these into R). Merge these values into your big dataset. Note: these values may be repeated for each state.



In [123]:
def revenue(coffee_co):
    rev = {'Starbucks': 32250000000, "Dunkin' Donuts": 1370000000, "Peet's Coffee & Tea": 983000000, "Tim Horton's": 3340000000, "Panera Bread": 5795000000, "Caribou Coffee": 262000000, "Au Bon Pain": 351000000, "The Coffee Bean & Tea Leaf": 500000000, "McDonald's": 23223000000}
    return rev.get(coffee_co)

In [124]:
combined['Revenue'] = combined['Coffee Chain'].apply(revenue)

In [125]:
combined

,State,Coffee Chain,Location Count,State Abbreviation,Population,Revenue
0,Alaska,Starbucks,24,AK,733391,32250000000
1,Alabama,Starbucks,73,AL,5024279,32250000000
2,Arkansas,Starbucks,33,AR,3011524,32250000000
3,Arizona,Starbucks,279,AZ,7151502,32250000000
4,California,Starbucks,2362,CA,39538223,32250000000
...,...,...,...,...,...,...
327,Vermont,McDonald's,30,VT,643077,23223000000
328,Washington,McDonald's,326,WA,7705281,23223000000
329,Wisconsin,McDonald's,353,WI,5893718,23223000000
330,West Virginia,McDonald's,107,WV,1793716,23223000000


6. Create a region variable in your dataset according to the scheme on this wikipedia page: Northeast, Midwest, South, West. You do not need to scrape this information.

In [126]:
def region(state_name):  
    region = {
    "Connecticut": "Northeast",
    "Maine": "Northeast",
    "Massachusetts": "Northeast",
    "New Hampshire": "Northeast",
    "Rhode Island": "Northeast",
    "Vermont": "Northeast",
    "New Jersey": "Northeast",
    "New York": "Northeast",
    "Pennsylvania": "Northeast",
    "Illinois": "Midwest",
    "Indiana": "Midwest",
    "Michigan": "Midwest",
    "Ohio": "Midwest",
    "Wisconsin": "Midwest",
    "Iowa": "Midwest",
    "Kansas": "Midwest",
    "Minnesota": "Midwest",
    "Missouri": "Midwest",
    "Nebraska": "Midwest",
    "North Dakota": "Midwest",
    "South Dakota": "Midwest",
    "Delaware": "South",
    "Florida": "South",
    "Georgia": "South",
    "Maryland": "South",
    "North Carolina": "South",
    "South Carolina": "South",
    "Virginia": "South",
    "Washington, D.C.": "South",
    "West Virginia": "South",
    "Alabama": "South",
    "Kentucky": "South",
    "Mississippi": "South",
    "Tennessee": "South",
    "Arkansas": "South",
    "Louisiana": "South",
    "Oklahoma": "South",
    "Texas": "South",
    "Arizona": "West",
    "Colorado": "West",
    "Idaho": "West",
    "Montana": "West",
    "Nevada": "West",
    "New Mexico": "West",
    "Utah": "West",
    "Wyoming": "West",
    "Alaska": "West",
    "California": "West",
    "Hawaii": "West",
    "Oregon": "West",
    "Washington": "West"}
    return region.get(state_name)


In [127]:
combined['Region'] = combined['State'].apply(region)

In [128]:
combined.head()

,State,Coffee Chain,Location Count,State Abbreviation,Population,Revenue,Region
0,Alaska,Starbucks,24,AK,733391,32250000000,West
1,Alabama,Starbucks,73,AL,5024279,32250000000,South
2,Arkansas,Starbucks,33,AR,3011524,32250000000,South
3,Arizona,Starbucks,279,AZ,7151502,32250000000,West
4,California,Starbucks,2362,CA,39538223,32250000000,West


Assess and comment on the prevalence of each chain. Some questions to consider (you don’t need to answer all of these and you may come up with your own):
Are some of these chains more prevalent in certain states than others? Possibly despite having less stores overall? Same questions for regions instead of states.

How does your chosen financial metric change by state and region for each chain? For example, having 5 stores in California is very different from having 5 stores in Wyoming.

Does the distribution of each chain’s stores match population distribution, by both state/region?

Do the financial data match what you’d expect based on the number and locations of the stores? Why or why not?

Automate: Convert your code for Exercises 1-3 above to a function that takes a single argument: the menuism URL. This function should

Scrape the information on state names and corresponding number of store locations on the webpage specified

Extract the name of the company from either the URL specified or the webpage

Return a clean, organized dataset that has three columns: state abbreviation, location count, company name. Company name will likely be repeated on every row.

In [129]:
def coffee_scraper(URL):
    for i in ['Starbucks', "Dunkin' Donuts", "Peet's Coffee & Tea", "Tim Horton's", 'Panera Bread', 'Caribou Coffee', 'Au Bon Pain','The Coffee Bean & Tea Leaf', "McDonald's"]:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        soup.find_all('div',{"class": "a"})[0]